# Exploratory Data Analysis (EDA)

In [ ]:
import pandas as pd
import os
from datetime import datetime, timedelta
from textwrap import dedent
import re
import warnings
from collections import defaultdict

warnings.filterwarnings("ignore")

##  - Parte 1

### 📌 Objetivo

En esta primera etapa del EDA, se busca consolidar todos los archivos `.csv` disponibles en una única estructura de datos y realizar un análisis de **completitud** para evaluar la calidad temporal de los registros de consumo eléctrico.


### 🗂️ 1. Concatenación de Archivos

Se cargan y concatenan todos los archivos individuales que contienen datos horarios de consumo eléctrico por vivienda. Esta etapa genera un único DataFrame llamado `merged_df`, que será la base para futuros análisis y modelos.

El resultado se guarda en `../data/vertical_raw_data_2.csv`.

### 📊 2. Análisis de Completitud de Datos

Por cada archivo original se verifica:
- Si las fechas en el nombre del archivo coinciden con los datos reales.
- Si existen **fechas faltantes** dentro del rango temporal.
- Si hay **días incompletos** (menos de 24 horas).
- Si hay **errores** de lectura o formato.

Se genera:
- Un reporte de texto con los resultados para cada archivo.
- Gráficos visuales por archivo que muestran la completitud diaria y mensual.
- Un resumen estadístico al finalizar el procesamiento.

Los resultados se guardan en `../data_analysis_results/`.

## 🔧 Limpieza de días incompletos

Además del análisis, se realizan acciones de limpieza para mejorar la calidad del dataset:
- Si un día tiene menos de 24 registros, se **rellenan** las horas faltantes con `ffill` o `bfill`.
- Si un día tiene **más de 24 registros**, se eliminan los duplicados conservando los primeros por hora.


In [ ]:
folder_path = "../data_raw/"
output_dir = "../data_analysis_results/"
merged_output_path = "../data/vertical_raw_data_2.csv"
os.makedirs(output_dir, exist_ok=True)

summary_stats = {
    'total_files': 0,
    'files_with_date_mismatch': 0,
    'files_with_missing_dates': 0,
    'files_with_incomplete_days': 0,
    'files_with_errors': 0,
    'cups_mismatches': defaultdict(int)
}

all_dataframes = []

In [3]:
def extract_dates_from_filename(filename):
    date_matches = re.findall(r'(\d{2}-\d{2}-\d{4})', filename)
    if len(date_matches) >= 2:
        try:
            return (datetime.strptime(date_matches[0], "%d-%m-%Y").date(),
                    datetime.strptime(date_matches[1], "%d-%m-%Y").date())
        except ValueError:
            pass
    return None, None

def extract_cups_from_filename(filename):
    cups_match = re.match(r'^([a-f0-9]+)', filename)
    return cups_match.group(1) if cups_match else "DESCONOCIDO"

def fix_incomplete_days(df):
    fixed_rows = []
    for date, group in df.groupby('fecha'):
        if len(group) < 24:
            full_day = pd.DataFrame({'hora': list(range(24))})
            merged = full_day.merge(group, on='hora', how='left')
            merged['fecha'] = date
            merged.sort_values(by='hora', inplace=True)
            merged.ffill(inplace=True)
            merged.bfill(inplace=True)
            fixed_rows.append(merged)
        elif len(group) > 24:
            trimmed = group.sort_values(by='hora').head(24)
            fixed_rows.append(trimmed)
        else:
            fixed_rows.append(group)
    return pd.concat(fixed_rows, ignore_index=True)


A su vez voy a preprocesar todas las columnas que quiero del dataset para que todas sigan un formato específico. Convertiré las columnas de fecha a a otra string pero con el mismo formato (formato dd/mm/yyyy). Transformare las columans de consumo_KWh y hora a float e int respectivamente para facilitar el trabajo con los datos.

In [4]:
contador_modificaciones = {}
total_modificaciones = 0

def formatear_hora(hora, cups=None):
    global contador_modificaciones, total_modificaciones
    hora_original = hora
    hora = str(hora).strip()
    modificado = False
    
    # Convertir '1:00' a '01:00', etc.
    if len(hora.split(':')[0]) == 1:
        hora = '0' + hora
        modificado = True
    
    # Eliminar '24:00:00' y '24:00' a '00:00' (formato más internacional)
    if hora == '24:00:00' or hora == '24:00':
        hora = '00:00'
    
    # Si se modificó y hay un CUPS, actualizamos el contador
    if modificado and cups is not None:
        contador_modificaciones[cups] = contador_modificaciones.get(cups, 0) + 1
        total_modificaciones += 1
        print(f"Advertencia: Hora modificada de '{hora_original}' a '{hora}' para el CUPS: {cups}")
    
    return hora

def limpiar_consumo(consumo):
    try:
        consumo = str(consumo).replace(',', '.')
        return float(consumo)
    except ValueError:
        print("HUBO ERROR")
        return float('nan')


In [ ]:
def generate_report(file):
    cups = extract_cups_from_filename(file)
    file_path = os.path.join(folder_path, file)
    report_name = os.path.splitext(file)[0] + "_report.txt"
    report_path = os.path.join(output_dir, report_name)

    with open(report_path, 'w', encoding='utf-8') as f:
        f.write(dedent(f"""\
        ====================================
        ANÁLISIS DE COMPLETITUD DE DATOS
        Archivo: {file}
        CUPS: {cups}
        ====================================
        """))

        try:
            df = pd.read_csv(file_path, sep=';', dtype={"cups": str})

            df['fecha'] = pd.to_datetime(df['fecha'], format='mixed', dayfirst=True, errors='coerce').dt.strftime('%d/%m/%Y')
            df['fecha'] = pd.to_datetime(df['fecha'], format='%d/%m/%Y').dt.date
            df['consumo_kWh'] = df['consumo_kWh'].apply(limpiar_consumo)

            df['hora'] = df.apply(
                lambda row: formatear_hora(row['hora'], cups=row['cups'] if 'cups' in df.columns else None),
                axis=1)
            # Agregar sección de resumen de modificaciones al reporte
            f.write(dedent(f"""\
            
            RESUMEN DE FORMATEO DE HORA:
            ====================================
            Modificaciones por CUPS: {contador_modificaciones.get(cups, 0)}
            ====================================
            """))
            df['hora'] = pd.to_datetime(df['hora'], format='%H:%M', errors='coerce').dt.hour

            expected_start, expected_end = extract_dates_from_filename(file)
            if expected_start is None:
                raise ValueError("No se pudieron extraer fechas del nombre del archivo")

            min_date = df['fecha'].min()
            max_date = df['fecha'].max()
            
            start_mismatch = min_date != expected_start
            end_mismatch = max_date != expected_end
            
            if start_mismatch or end_mismatch:
                summary_stats['files_with_date_mismatch'] += 1
                summary_stats['cups_mismatches'][cups] += 1

                start_diff = (min_date - expected_start).days if start_mismatch else 0
                end_diff = (expected_end - max_date).days if end_mismatch else 0
                
                f.write(dedent(f"""\
                ⚠️ DISCREPANCIA DETECTADA:
                • Esperado (según nombre): {expected_start} a {expected_end}
                • Encontrado en datos:    {min_date} a {max_date}
                • Días de desfase:
                  - Inicio: {abs(start_diff)} días ({'antes' if start_diff < 0 else 'después'})
                  - Fin: {abs(end_diff)} días ({'antes' if end_diff > 0 else 'después'})
                """))
            else:
                f.write("✅ El rango coincide con lo esperado\n")

            all_dates = pd.date_range(start=min_date, end=max_date, freq='D')
            missing_dates = all_dates[~all_dates.isin(df['fecha'])]
            if len(missing_dates) > 0:
                summary_stats['files_with_missing_dates'] += 1
                f.write(f"\n Fechas faltantes ({len(missing_dates)}):\n")
                for date in missing_dates:
                    f.write(f"  - {date.date()}\n")
            else:
                f.write("\n✅ No hay fechas faltantes en el rango.\n")

            hours_per_day = df.groupby('fecha')['hora'].count()
            incomplete_days = hours_per_day[(hours_per_day != 24)]
            if len(incomplete_days) > 0:
                summary_stats['files_with_incomplete_days'] += 1
                f.write(f"\n⚠️ Días con horas incompletas ({len(incomplete_days)}):\n")
                for date, count in incomplete_days.items():
                    f.write(f"  - {date}: {count}/24 horas\n")
            else:
                f.write("\n✅ Todos los días tienen 24 registros horarios.\n")

            total_days = len(all_dates)
            complete_days = total_days - len(missing_dates) - len(incomplete_days)
            completeness_percentage = (complete_days / total_days) * 100

            f.write("\n[RESUMEN DE COMPLETITUD]\n")
            f.write(f"• Total de días esperados: {total_days}\n")
            f.write(f"• Días completos (24 horas): {complete_days}\n")
            f.write(f"• Días incompletos: {len(incomplete_days)}\n")
            f.write(f"• Días faltantes: {len(missing_dates)}\n")
            f.write(f"• Completitud: {completeness_percentage:.2f}%\n")

            df_fixed = fix_incomplete_days(df)
            all_dataframes.append(df_fixed)

        except Exception as e:
            summary_stats['files_with_errors'] += 1
            f.write(f"\n❌ ERROR EN EL PROCESAMIENTO:\n{str(e)}\n")

In [6]:
for file in os.listdir(folder_path):
    if file.endswith(".csv"):
        summary_stats['total_files'] += 1
        print(f"Procesando: {file}")
        generate_report(file)

merged_df = pd.concat(all_dataframes, ignore_index=True)
merged_df.to_csv(merged_output_path, index=False, sep=";")

Procesando: 0114f2a9ef8e Consumo 11-08-2021_11-08-2023.csv
Procesando: 011952cf0a27 Consumo 26-08-2021_26-08-2023.csv
Procesando: 02b3106d73c7 Consumo 17-08-2021_17-08-2023.csv
Procesando: 03c8338d7f1d Consumo 07-08-2021_07-08-2023.csv
Procesando: 04666163609d Consumo 07-08-2021_07-08-2023.csv
Procesando: 07efa654691a Consumo 07-08-2021_07-08-2023.csv
Procesando: 08565e582514 Consumo 07-08-2021_07-08-2023.csv
Procesando: 0d9678df292f Consumo 07-08-2021_07-08-2023.csv
Procesando: 0e6a45c8d43b Consumo 07-08-2021_07-08-2023.csv
Procesando: 11c65598e4aa Consumo 30-08-2021_30-08-2023.csv
Procesando: 127a14c3f85b Consumo 07-08-2021_07-08-2023.csv
Procesando: 17a509e5b3fd Consumo 24-08-2021_24-08-2023.csv
Procesando: 19153be7ac88 Consumo 07-08-2021_07-08-2023.csv
Procesando: 1a0b9aa571b5 Consumo 30-08-2021_30-08-2023.csv
Procesando: 1bd2d65d3ee4 Consumo 24-08-2021_24-08-2023.csv
Procesando: 1c8a857523c6 Consumo 07-08-2021_07-08-2023.csv
Procesando: 20eff1413834 Consumo 07-08-2021_07-08-2023.c

Resumen general del tratamiento de datos:

In [7]:
print("\n--- Resumen de modificaciones ---")
print(f"Total de horas modificadas: {total_modificaciones}")
for cups, count in contador_modificaciones.items():
    print(f"CUPS {cups}: {count} veces")


--- Resumen de modificaciones ---
Total de horas modificadas: 6534
CUPS c1244d6dea7a: 6534 veces


In [8]:
print("\nResumen de análisis:")
print(f"Total de archivos procesados: {summary_stats['total_files']}")
print(f"Archivos con discrepancias de fecha: {summary_stats['files_with_date_mismatch']}")
print(f"Archivos con fechas faltantes: {summary_stats['files_with_missing_dates']}")
print(f"Archivos con días incompletos: {summary_stats['files_with_incomplete_days']}")
print(f"Archivos con errores: {summary_stats['files_with_errors']}")
print(f"\nAnálisis completado. Resultados guardados en: {output_dir}")


Resumen de análisis:
Total de archivos procesados: 131
Archivos con discrepancias de fecha: 130
Archivos con fechas faltantes: 0
Archivos con días incompletos: 130
Archivos con errores: 0

Análisis completado. Resultados guardados en: data_analysis_results/


In [ ]:
def generate_summary(output_dir, summary_stats):
    summary_path = os.path.join(output_dir, "summary.txt")
    
    with open(summary_path, 'w', encoding='utf-8') as f:
        f.write("=== RESUMEN DE MODIFICACIONES ===\n\n")
        f.write(f"Total de horas modificadas: {total_modificaciones}\n")
        for cups, count in contador_modificaciones.items():
            f.write(f"CUPS {cups}: {count} veces\n")
        
        f.write("\n=== RESUMEN DE ANÁLISIS ===\n\n")
        f.write(f"Total de archivos procesados: {summary_stats['total_files']}\n")
        f.write(f"Archivos con discrepancias de fecha: {summary_stats['files_with_date_mismatch']}\n")
        f.write(f"Archivos con fechas faltantes: {summary_stats['files_with_missing_dates']}\n")
        f.write(f"Archivos con días incompletos: {summary_stats['files_with_incomplete_days']}\n")
        f.write(f"Archivos con errores: {summary_stats['files_with_errors']}\n")
        f.write(f"\nAnálisis completado. Resultados guardados en: {output_dir}\n")



generate_summary(output_dir, summary_stats)